<a href="https://colab.research.google.com/github/jpan2575/Homework4_Pan/blob/main/Homework4_Pan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Pandas
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

filedir = '/content/drive/MyDrive/FWE458/'

fname = filedir + 'Wcr_GPPdaily.csv'

#Dataframe
df = pd.read_csv(fname)

print(df)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
      TIMESTAMP    TA_F  SW_IN_F  VPD_F  GPP_NT_VUT_REF
0      19990101 -20.063   72.603  0.413       -0.517364
1      19990102 -12.814   12.358  0.147       -0.094241
2      19990103 -12.625   33.132  0.128       -0.166819
3      19990104 -18.652   93.481  0.263       -0.582301
4      19990105 -20.269   45.502  0.261       -0.568240
...         ...     ...      ...    ...             ...
5839   20141227  -5.112   22.956  0.497        0.195777
5840   20141228 -12.408   33.332  0.531        0.093025
5841   20141229 -18.204   69.597  0.461       -0.234488
5842   20141230 -22.252   81.959  0.448       -0.315846
5843   20141231 -19.436   42.164  0.619       -0.217537

[5844 rows x 5 columns]


In [13]:
# Drop the TIMESTAMP column
df.drop(columns=['TIMESTAMP'], inplace=True)
df.head()

,TA_F,SW_IN_F,VPD_F,GPP_NT_VUT_REF
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.568240


In [17]:
#Rename columns
df = df.rename(columns={'TA_F': 'TA', 'SW_IN_F': 'SW', 'VPD_F': 'VPD','GPP_NT_VUT_REF':'GPP'})
df.head()

,TA,SW,VPD,GPP
0,-20.063,72.603,0.413,-0.517364
1,-12.814,12.358,0.147,-0.094241
2,-12.625,33.132,0.128,-0.166819
3,-18.652,93.481,0.263,-0.582301
4,-20.269,45.502,0.261,-0.568240


In [18]:
#Check for missing values
DF.isnull().sum()

,0
TIMESTAMP,0
TA_F,0
SW_IN_F,0
VPD_F,0
GPP_NT_VUT_REF,0


In [31]:
# Interaction terms
df['SW*VPD'] = df['SW'] * df['VPD']
df['SW*TA'] = df['SW'] * df['TA']
df['VPD*TA'] = df['VPD'] * df['TA']
#SW*VPD: Interaction between SW and VPD
#SW*TA: Interaction between SW and TA
#VPD*TA: Interaction between VPD and TA
df.head()

,TA,SW,VPD,GPP,SW_VPD,SW_TA,VPD_TA,SW*VPD,SW*TA,VPD*TA
0,-20.063,72.603,0.413,-0.517364,29.985039,-1456.633989,-8.286019,29.985039,-1456.633989,-8.286019
1,-12.814,12.358,0.147,-0.094241,1.816626,-158.355412,-1.883658,1.816626,-158.355412,-1.883658
2,-12.625,33.132,0.128,-0.166819,4.240896,-418.291500,-1.616000,4.240896,-418.291500,-1.616000
3,-18.652,93.481,0.263,-0.582301,24.585503,-1743.607612,-4.905476,24.585503,-1743.607612,-4.905476
4,-20.269,45.502,0.261,-0.568240,11.876022,-922.280038,-5.290209,11.876022,-922.280038,-5.290209


In [51]:
from scipy.optimize import minimize


def linear_model(beta, X):
    # Extract coefficients
    b0, b1, b2, b3, b4, b5, b6 = beta

    # Calculate equation
    GPP_pred = b0 + b1 * X['SW'] + b2 * X['VPD'] + b3 * X['TA'] + \
               b4 * X['SW*VPD'] + b5 * X['SW*TA'] + b6 * X['VPD*TA']

    return GPP_pred

#MSE
def mse(beta, X, y):
    y_pred = np.dot(X, beta)
    error = y - y_pred
    MSE = np.mean(error**2)
    return MSE

X = df[['SW', 'VPD', 'TA', 'SW*VPD', 'SW*TA', 'VPD*TA']]
y = df['GPP']

#Predict, not sure if need to include B0
initial_beta = np.zeros(6)

# Minimize the MSE
result = minimize(mse, initial_beta, args=(X, y), method='BFGS')

# Extract optimized coefficients
optimized_beta = result.x
print("Optimized Coefficients (β0, β1, β2, β3, β4, β5, β6):", optimized_beta)

Optimized Coefficients (β0, β1, β2, β3, β4, β5, β6): [-8.27226284e+00 -1.00057966e+03  1.13499168e+02  3.27870503e+00
 -5.39227579e-01  1.24226938e+01]


In [39]:
from sklearn.linear_model import LinearRegression

# Fit the linear model
model = LinearRegression()
model.fit(X, y)

# Extract coefficients
coefficients = model.coef_
intercept = model.intercept_

# Print results
print("Intercept:", intercept) #bias
print("Coefficients:", coefficients) #slope

Intercept: -944.0672018203164
Coefficients: [-3.24914968e+00 -6.48574173e+02  8.21699813e+01  1.79249151e+00
 -4.56050357e-01  1.35954025e+01]


In [49]:
#Scipy R^2
intercept_scipy = np.mean(y) - np.dot(optimized_beta, np.mean(X, axis=0))
y_pred_scipy = intercept_scipy + np.dot(X, optimized_beta)
r2_scipy = r2_score(y, y_pred_scipy)

# Print results
print(f"R-squared (R²): {r2_scipy:.4f}")

R-squared (R²): 0.0070


In [50]:
Sklearn R^2
from sklearn.metrics import r2_score

# Compute predictions using sklearn model
y_pred_sklearn = model.predict(X)  # Use the entire DataFrame X for prediction

# Compute R-squared for sklearn model
r2_sklearn = r2_score(y, y_pred_sklearn)

print(f"R-squared (R²): {r2_sklearn:.4f}")

R-squared (R²): 0.0136


The Sci py is found utilizing the residual/total sum of square which you need to find while sklearn you can just use the function. We find that the sklearn has a R^2 closer to 0 which means that the variance in the data cannot be explained as well as the scipyt R^2, but for the most part are relatively similar.